# Etap 2: Data Load and Process

## Overview
This notebook implements Phase 1 of the Etap 2 analysis plan: basic data loading, initial processing, and pickle file creation for the new data batch.

## Objectives
- Load and clean listings.csv, calendar.csv, and reviews.csv
- Create standardized pickle files for efficient subsequent analysis
- Perform basic validation and data integrity checks
- Document data quality improvements compared to Etap 1

In [21]:
import os
import pandas as pd
import numpy as np
import logging
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Define paths for Etap 2 data
listings_path_e2 = '../data/raw2/listings.csv'
calendar_path_e2 = '../data/raw2/calendar.csv'
reviews_path_e2 = '../data/raw2/reviews.csv'

# Define paths for processed (pickled) data
processed_data_dir_e2 = '../data/processed/etap2/'
listings_pkl_path_e2 = os.path.join(processed_data_dir_e2, 'listings_e2_df.pkl')
calendar_pkl_path_e2 = os.path.join(processed_data_dir_e2, 'calendar_e2_df.pkl')
reviews_pkl_path_e2 = os.path.join(processed_data_dir_e2, 'reviews_e2_df.pkl')

# Create processed data directory if it doesn't exist
os.makedirs(processed_data_dir_e2, exist_ok=True)
print(f"Processed data directory '{processed_data_dir_e2}' ensured.")

# Define validation schemas
EXPECTED_COLUMNS = {
    'listings': ['id', 'name', 'host_id', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'price'],
    'calendar': ['listing_id', 'date', 'available', 'price'],
    'reviews': ['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']
}

MIN_EXPECTED_ROWS = {
    'listings': 10000,
    'calendar': 100000,
    'reviews': 50000
}

print("✓ Setup completed")

Processed data directory '../data/processed/etap2/' ensured.
✓ Setup completed


In [22]:
def check_file_freshness(csv_path, pickle_path):
    """Check if pickle file is newer than CSV file."""
    if not os.path.exists(csv_path) or not os.path.exists(pickle_path):
        return False
    return os.path.getmtime(pickle_path) > os.path.getmtime(csv_path)

def validate_dataframe(df, name, expected_cols, min_rows):
    """Validate dataframe structure and content."""
    validation = {
        'valid': True,
        'warnings': [],
        'errors': []
    }

    # Check minimum rows
    if len(df) < min_rows:
        validation['warnings'].append(f"Only {len(df)} rows, expected at least {min_rows}")

    # Check for expected columns
    missing_cols = [col for col in expected_cols if col not in df.columns]
    if missing_cols:
        validation['warnings'].append(f"Missing expected columns: {missing_cols}")

    # Check for completely empty columns
    empty_cols = [col for col in df.columns if df[col].isna().all()]
    if empty_cols:
        validation['warnings'].append(f"Completely empty columns: {empty_cols}")

    # Check for duplicate rows
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        validation['warnings'].append(f"{duplicates} duplicate rows found")

    return validation

def get_missing_value_summary(df, name):
    """Get comprehensive missing value summary."""
    missing_counts = df.isnull().sum()
    missing_pct = (missing_counts / len(df)) * 100

    summary = pd.DataFrame({
        'count': missing_counts,
        'percentage': missing_pct
    })

    return summary[summary['count'] > 0].sort_values('percentage', ascending=False)

print("✓ Utility functions defined")

✓ Utility functions defined


## 1. Load and Process Listings Data

In [15]:
import os
import pandas as pd

# Define your logger, EXPECTED_COLUMNS, MIN_EXPECTED_ROWS, listings_path_e2, and listings_pkl_path_e2 before this code

def load_and_process_listings(csv_path, pickle_path, force_reload=False):
    """Load and process listings data with comprehensive validation."""

    # Check if we should use pickle
    use_pickle = (os.path.exists(pickle_path) and
                  check_file_freshness(csv_path, pickle_path) and
                  not force_reload)

    try:
        if use_pickle:
            df = pd.read_pickle(pickle_path)
            logger.info(f"Loaded listings from pickle: {pickle_path}")
            loaded_from_pickle = True
        else:
            logger.info(f"Loading listings from CSV: {csv_path}")
            df = pd.read_csv(csv_path, low_memory=False)

            # Basic data cleaning
            if 'price' in df.columns:
                # Clean price column if it exists
                df['price_cleaned'] = df['price'].astype(str).str.replace(r'[\$,]', '', regex=True)
                df['price_cleaned'] = pd.to_numeric(df['price_cleaned'], errors='coerce')

            loaded_from_pickle = False
            logger.info(f"Loaded {len(df)} rows from CSV")

        # Validate data
        validation = validate_dataframe(df, 'listings', EXPECTED_COLUMNS['listings'], MIN_EXPECTED_ROWS['listings'])

        # Save to pickle if loaded from CSV and validation passed
        if not loaded_from_pickle and validation['valid']:
            df.to_pickle(pickle_path)
            logger.info(f"Saved listings to pickle: {pickle_path}")

        return df, validation, loaded_from_pickle

    except Exception as e:
        logger.error(f"Error loading listings: {e}")
        raise

# Load listings data
print("="*50)
print("LOADING LISTINGS DATA")
print("="*50)

try:
    listings_e2_df, listings_validation, loaded_from_pickle = load_and_process_listings(
        listings_path_e2, listings_pkl_path_e2
    )
    print(f"\n✓ Listings loaded successfully: {listings_e2_df.shape}")
    print(f"  Source: {'Pickle' if loaded_from_pickle else 'CSV'}")
    print(f"  Columns: {list(listings_e2_df.columns)[:10]}..." if len(listings_e2_df.columns) > 10 else f"  Columns: {list(listings_e2_df.columns)}")

    # Show validation results
    if listings_validation['warnings']:
        print("  Warnings:", "; ".join(listings_validation['warnings']))
    if listings_validation['errors']:
        print("  Errors:", "; ".join(listings_validation['errors']))

    # Quick data preview
    print("\nData Preview:")
    print(listings_e2_df.head(3))

except Exception as e:
    print(f"✗ Failed to load listings: {e}")
    listings_e2_df = None

2025-06-26 03:07:22,702 - INFO - Loaded listings from pickle: ../data/processed/etap2/listings_e2_df.pkl


LOADING LISTINGS DATA

✓ Listings loaded successfully: (47572, 75)
  Source: Pickle
  Columns: ['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id']...
  Warnings: Completely empty columns: ['neighbourhood_group_cleansed', 'calendar_updated', 'license']

Data Preview:
                    id                                      listing_url  \
0             18974065             https://www.nocarz.pl/rooms/18974065   
1  1074535877676155362  https://www.nocarz.pl/rooms/1074535877676155362   
2   796304692597746009   https://www.nocarz.pl/rooms/796304692597746009   

        scrape_id last_scraped       source  \
0  20241211032909   2024-12-11  city scrape   
1  20241211032909   2024-12-11  city scrape   
2  20241211032909   2024-12-11  city scrape   

                                    name  \
0  Charming split-level one bedroom flat   
1             Stunning two bed apartment   
2                  Southall,

In [16]:
loaded_calendar_from_pickle = False
try:
    if os.path.exists(calendar_pkl_path_e2):
        calendar_e2_df = pd.read_pickle(calendar_pkl_path_e2)
        print(f"calendar_e2_df loaded successfully from pickle file: {calendar_pkl_path_e2}")
        loaded_calendar_from_pickle = True
    else:
        print(f"Pickle file not found: {calendar_pkl_path_e2}. Loading from CSV and processing...")
        cols_to_load_calendar = ['listing_id', 'date', 'available', 'price']
        calendar_e2_df = pd.read_csv(calendar_path_e2, usecols=cols_to_load_calendar)
        print(f"{calendar_path_e2} loaded successfully.")

        # Process 'available'
        if 'available' in calendar_e2_df.columns:
            calendar_e2_df['available'] = calendar_e2_df['available'].map({'t': True, 'f': False}).astype(bool)

        # Process 'price' into 'price_cleaned'
        if 'price' in calendar_e2_df.columns and calendar_e2_df['price'].dtype == 'object':
            calendar_e2_df['price_cleaned'] = calendar_e2_df['price'].replace({'[\\$,]': ''}, regex=True).astype(float)

        # Process 'date'
        if 'date' in calendar_e2_df.columns:
            calendar_e2_df['date'] = pd.to_datetime(calendar_e2_df['date'])
        print("Essential processing for calendar_e2_df completed.")

    print("\nHead of calendar_e2_df:")
    print(calendar_e2_df.head())
    print("\nInfo for calendar_e2_df:")
    calendar_e2_df.info()
    print(f"\nNumber of duplicate rows in calendar_e2_df: {calendar_e2_df.duplicated().sum()}")

except Exception as e:
    print(f"Error loading or processing calendar_e2_df: {e}")

def load_and_process_calendar(csv_path, pickle_path, force_reload=False):
    """Load and process calendar data with optimizations."""

    use_pickle = (os.path.exists(pickle_path) and
                  check_file_freshness(csv_path, pickle_path) and
                  not force_reload)

    try:
        if use_pickle:
            df = pd.read_pickle(pickle_path)
            logger.info(f"Loaded calendar from pickle: {pickle_path}")
            loaded_from_pickle = True
        else:
            logger.info(f"Loading calendar from CSV: {csv_path}")
            # Load only essential columns for memory efficiency
            cols_to_load = ['listing_id', 'date', 'available', 'price']
            df = pd.read_csv(csv_path, usecols=cols_to_load)

            # Process data
            if 'available' in df.columns:
                df['available'] = df['available'].map({'t': True, 'f': False}).astype(bool)

            if 'price' in df.columns and df['price'].dtype == 'object':
                df['price_cleaned'] = df['price'].str.replace(r'[\$,]', '', regex=True)
                df['price_cleaned'] = pd.to_numeric(df['price_cleaned'], errors='coerce')

            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'], errors='coerce')

            loaded_from_pickle = False
            logger.info(f"Processed {len(df)} calendar rows")

        # Validate
        validation = validate_dataframe(df, 'calendar', EXPECTED_COLUMNS['calendar'], MIN_EXPECTED_ROWS['calendar'])

        # Check for date range issues
        if 'date' in df.columns:
            date_range = df['date'].agg(['min', 'max'])
            logger.info(f"Calendar date range: {date_range['min']} to {date_range['max']}")

            # Check for missing dates per listing
            listing_date_counts = df.groupby('listing_id')['date'].count()
            short_periods = (listing_date_counts < 30).sum()
            if short_periods > 0:
                validation['warnings'].append(f"{short_periods} listings with <30 days of data")

        if not loaded_from_pickle and validation['valid']:
            df.to_pickle(pickle_path)
            logger.info(f"Saved calendar to pickle: {pickle_path}")

        return df, validation, loaded_from_pickle

    except Exception as e:
        logger.error(f"Error loading calendar: {e}")
        raise

# Load calendar data
print("\n" + "="*50)
print("LOADING CALENDAR DATA")
print("="*50)

try:
    calendar_e2_df, calendar_validation, loaded_from_pickle = load_and_process_calendar(
        calendar_path_e2, calendar_pkl_path_e2
    )
    print(f"\n✓ Calendar loaded successfully: {calendar_e2_df.shape}")
    print(f"  Source: {'Pickle' if loaded_from_pickle else 'CSV'}")
    print(f"  Columns: {list(calendar_e2_df.columns)}")

    # Show validation results
    if calendar_validation['warnings']:
        print("  Warnings:", "; ".join(calendar_validation['warnings']))
    if calendar_validation['errors']:
        print("  Errors:", "; ".join(calendar_validation['errors']))

    # Quick data preview
    print("\nData Preview:")
    print(calendar_e2_df.head(3))

except Exception as e:
    print(f"✗ Failed to load calendar: {e}")
    calendar_e2_df = None

calendar_e2_df loaded successfully from pickle file: ../data/processed/etap2/calendar_e2_df.pkl

Head of calendar_e2_df:
            listing_id       date  available    price  price_cleaned
0             19902160 2025-05-02      False   $37.00           37.0
1             37075766 2025-11-25      False   $72.00           72.0
2  1091268028307262502 2025-05-28       True  $311.00          311.0
3   829621911605471263 2025-12-03       True   $36.00           36.0
4              4223187 2025-09-16      False   $86.00           86.0

Info for calendar_e2_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17363102 entries, 0 to 17363101
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   listing_id     int64         
 1   date           datetime64[ns]
 2   available      bool          
 3   price          object        
 4   price_cleaned  float64       
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(1)
memory usage

2025-06-26 03:07:32,945 - INFO - Loaded calendar from pickle: ../data/processed/etap2/calendar_e2_df.pkl
2025-06-26 03:07:38,052 - INFO - Calendar date range: 2024-12-11 00:00:00 to 2025-12-21 00:00:00
2025-06-26 03:07:38,052 - INFO - Calendar date range: 2024-12-11 00:00:00 to 2025-12-21 00:00:00



✓ Calendar loaded successfully: (17363102, 5)
  Source: Pickle
  Columns: ['listing_id', 'date', 'available', 'price', 'price_cleaned']

Data Preview:
            listing_id       date  available    price  price_cleaned
0             19902160 2025-05-02      False   $37.00           37.0
1             37075766 2025-11-25      False   $72.00           72.0
2  1091268028307262502 2025-05-28       True  $311.00          311.0


## 3. Load and Process Reviews Data

In [20]:
## 4. Cross-Dataset Validation & Summary









































































    listings_e2_df = None    print(f"✗ Failed to load listings: {e}")except Exception as e:        print(missing_summary.head(10))        print("\nTop Missing Values:")    if not missing_summary.empty:    missing_summary = get_missing_value_summary(listings_e2_df, 'listings')    # Missing value summary        print(listings_e2_df.head(3))    print("\nData Preview:")    # Quick data preview        print("  Errors:", "; ".join(listings_validation['errors']))    if listings_validation['errors']:        print("  Warnings:", "; ".join(listings_validation['warnings']))    if listings_validation['warnings']:    # Show validation results    print(f"  Columns: {list(listings_e2_df.columns)[:10]}..." if len(listings_e2_df.columns) > 10 else f"  Columns: {list(listings_e2_df.columns)}")    print(f"  Source: {'Pickle' if loaded_from_pickle else 'CSV'}")    print(f"\n✓ Listings loaded successfully: {listings_e2_df.shape}")    )        listings_path_e2, listings_pkl_path_e2    listings_e2_df, listings_validation, loaded_from_pickle = load_and_process_listings(try:print("="*50)print("LOADING LISTINGS DATA")print("="*50)# Load listings data        raise        logger.error(f"Error loading listings: {e}")    except Exception as e:        return df, validation, loaded_from_pickle            logger.info(f"Saved listings to pickle: {pickle_path}")            df.to_pickle(pickle_path)































    reviews_e2_df = None    print(f"✗ Failed to load reviews: {e}")except Exception as e:        print(missing_summary.head(5))        print("\nTop Missing Values:")    if not missing_summary.empty:    missing_summary = get_missing_value_summary(reviews_e2_df, 'reviews')    # Missing value summary        print(reviews_e2_df.head(3))
    print("\nData Preview:")    # Quick data preview            print("  Errors:", "; ".join(reviews_validation['errors']))    if reviews_validation['errors']:        print("  Warnings:", "; ".join(reviews_validation['warnings']))
    if reviews_validation['warnings']:    # Show validation results        print(f"  Columns: {list(reviews_e2_df.columns)}")    print(f"  Source: {'Pickle' if loaded_from_pickle else 'CSV'}")    print(f"\n✓ Reviews loaded successfully: {reviews_e2_df.shape}")

    )        reviews_path_e2, reviews_pkl_path_e2    reviews_e2_df, reviews_validation, loaded_from_pickle = load_and_process_reviews(
try:print("="*50)print("LOADING REVIEWS DATA")print("\n" + "="*50)# Load reviews data        raise        logger.error(f"Error loading reviews: {e}")    except Exception as e:



            return df, validation, loaded_from_pickle                    logger.info(f"Saved reviews to pickle: {pickle_path}")




            df.to_pickle(pickle_path)        if not loaded_from_pickle and validation['valid']:        # Save to pickle if loaded from CSV and validation passed




            logger.info(f"Reviews date range: {date_range['min']} to {date_range['max']}")
            date_range = df['date'].agg(['min', 'max'])        if 'date' in df.columns:        # Check date range                validation = validate_dataframe(df, 'reviews', EXPECTED_COLUMNS['reviews'], MIN_EXPECTED_ROWS['reviews'])        # Validate                    logger.info(f"Processed {len(df)} review rows")
        if not loaded_from_pickle and validation['valid']:        # Save to pickle if loaded from CSV and validation passed

            loaded_from_pickle = False                            df['comment_length'] = df['comments'].str.len()
        validation = validate_dataframe(df, 'listings', EXPECTED_COLUMNS['listings'], MIN_EXPECTED_ROWS['listings'])                # Calculate comment length                df['comments'] = df['comments'].replace('nan', np.nan)

        # Validate data            logger.info(f"Loaded {len(df)} rows from CSV")
                # Replace 'nan' strings with actual NaN            loaded_from_pickle = False                df['price_cleaned'] = pd.to_numeric(df['price_cleaned'], errors='coerce')                df['price_cleaned'] = df['price'].astype(str).str.replace(r'[\$,]', '', regex=True)


                df['comments'] = df['comments'].astype(str).str.strip()
                # Remove excessive whitespace            if 'comments' in df.columns:            # Basic text cleaning for comments                            df['date'] = pd.to_datetime(df['date'], errors='coerce')

                # Clean price column if it exists            if 'price' in df.columns:            if 'date' in df.columns:            # Process date column

            # Basic data cleaning            df = pd.read_csv(csv_path, low_memory=False)            logger.info(f"Loading listings from CSV: {csv_path}")

                        df = pd.read_csv(csv_path)        else:            loaded_from_pickle = True            logger.info(f"Loaded listings from pickle: {pickle_path}")            df = pd.read_pickle(pickle_path)

            logger.info(f"Loading reviews from CSV: {csv_path}")
        else:
            loaded_from_pickle = True            logger.info(f"Loaded reviews from pickle: {pickle_path}")        if use_pickle:    try:                  not force_reload)                  check_file_freshness(csv_path, pickle_path) and    use_pickle = (os.path.exists(pickle_path) and    # Check if we should use pickle    """Load and process listings data with comprehensive validation."""def load_and_process_listings(csv_path, pickle_path, force_reload=False):




            df = pd.read_pickle(pickle_path)        if use_pickle:
    try:                      not force_reload)def load_and_process_reviews(csv_path, pickle_path, force_reload=False):

                  check_file_freshness(csv_path, pickle_path) and
    use_pickle = (os.path.exists(pickle_path) and    """Load and process reviews data with validation and text cleaning."""


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 128)

## 4. Cross-Dataset Validation & Summary

In [ ]:
def perform_cross_dataset_validation():
    """Perform validation across all three datasets."""

    print("\n" + "="*50)
    print("CROSS-DATASET VALIDATION")
    print("="*50)

    results = {
        'datasets_loaded': [],
        'id_overlaps': {},
        'join_success_rates': {},
        'total_unique_ids': {},
        'orphan_records': {}
    }

    # Check which datasets are available
    datasets = {
        'listings': listings_e2_df if 'listings_e2_df' in globals() and listings_e2_df is not None else None,
        'calendar': calendar_e2_df if 'calendar_e2_df' in globals() and calendar_e2_df is not None else None,
        'reviews': reviews_e2_df if 'reviews_e2_df' in globals() and reviews_e2_df is not None else None
    }

    for name, df in datasets.items():
        if df is not None:
            results['datasets_loaded'].append(name)
            print(f"✓ {name}: {df.shape}")
        else:
            print(f"✗ {name}: Not loaded")

    # ID Analysis
    if datasets['listings'] is not None:
        listings_ids = set(datasets['listings']['id'].dropna())
        results['total_unique_ids']['listings'] = len(listings_ids)
        print(f"\n📊 Unique listing IDs in listings.csv: {len(listings_ids):,}")

    if datasets['calendar'] is not None:
        calendar_ids = set(datasets['calendar']['listing_id'].dropna())
        results['total_unique_ids']['calendar'] = len(calendar_ids)
        print(f"📊 Unique listing IDs in calendar.csv: {len(calendar_ids):,}")

    if datasets['reviews'] is not None:
        reviews_ids = set(datasets['reviews']['listing_id'].dropna())
        results['total_unique_ids']['reviews'] = len(reviews_ids)
        print(f"📊 Unique listing IDs in reviews.csv: {len(reviews_ids):,}")

    # Join Analysis
    if datasets['listings'] is not None and datasets['calendar'] is not None:
        overlap = listings_ids.intersection(calendar_ids)
        results['id_overlaps']['listings_calendar'] = len(overlap)
        results['join_success_rates']['listings_calendar'] = len(overlap) / len(listings_ids) * 100

        orphan_calendar = len(calendar_ids - listings_ids)
        results['orphan_records']['calendar'] = orphan_calendar

        print(f"\n🔗 Listings-Calendar Join:")
        print(f"   Successful joins: {len(overlap):,} ({len(overlap)/len(listings_ids)*100:.1f}% of listings)")
        print(f"   Orphan calendar records: {orphan_calendar:,}")

    if datasets['listings'] is not None and datasets['reviews'] is not None:
        overlap = listings_ids.intersection(reviews_ids)
        results['id_overlaps']['listings_reviews'] = len(overlap)
        results['join_success_rates']['listings_reviews'] = len(overlap) / len(listings_ids) * 100

        orphan_reviews = len(reviews_ids - listings_ids)
        results['orphan_records']['reviews'] = orphan_reviews

        print(f"\n🔗 Listings-Reviews Join:")
        print(f"   Successful joins: {len(overlap):,} ({len(overlap)/len(listings_ids)*100:.1f}% of listings)")
        print(f"   Orphan review records: {orphan_reviews:,}")

    # Three-way join
    if all(df is not None for df in datasets.values()):
        three_way = listings_ids.intersection(calendar_ids).intersection(reviews_ids)
        results['id_overlaps']['all_three'] = len(three_way)

        print(f"\n🎯 Three-way Join (All datasets):")
        print(f"   Complete records: {len(three_way):,} listings")

    return results

# Perform validation
validation_results = perform_cross_dataset_validation()

In [ ]:
def generate_summary_report():
    """Generate comprehensive summary report."""

    print("\n" + "="*50)
    print("SUMMARY REPORT")
    print("="*50)

    # Data loading summary
    print("\n📋 Data Loading Summary:")
    for dataset in validation_results['datasets_loaded']:
        print(f"   ✓ {dataset.title()} data loaded successfully")

    # Key metrics comparison (vs Etap 1 findings)
    print("\n📊 Key Metrics (vs Etap 1 baseline):")

    if 'listings' in validation_results['total_unique_ids']:
        listings_count = validation_results['total_unique_ids']['listings']
        print(f"   • Total listings: {listings_count:,}")

    if 'all_three' in validation_results['id_overlaps']:
        complete_records = validation_results['id_overlaps']['all_three']
        print(f"   • Complete records (all 3 datasets): {complete_records:,} (Etap 1: ~11,000)")

    total_orphans = sum(validation_results['orphan_records'].values())
    if total_orphans > 0:
        print(f"   • Total orphan records: {total_orphans:,} (Etap 1: ~75,000)")

    # Critical missing values check
    if 'listings_e2_df' in globals() and listings_e2_df is not None:
        missing_neighbourhood = listings_e2_df['neighbourhood_cleansed'].isna().sum()
        missing_neighbourhood_pct = (missing_neighbourhood / len(listings_e2_df)) * 100
        print(f"   • Missing neighbourhood_cleansed: {missing_neighbourhood:,} ({missing_neighbourhood_pct:.1f}%) (Etap 1: ~30%)")

        # Check other critical features
        critical_features = ['bedrooms', 'bathrooms_text', 'beds', 'latitude', 'longitude']
        for feature in critical_features:
            if feature in listings_e2_df.columns:
                missing_count = listings_e2_df[feature].isna().sum()
                missing_pct = (missing_count / len(listings_e2_df)) * 100
                print(f"   • Missing {feature}: {missing_count:,} ({missing_pct:.1f}%)")

    # Calendar observation periods
    if 'calendar_e2_df' in globals() and calendar_e2_df is not None:
        listing_periods = calendar_e2_df.groupby('listing_id')['date'].count()
        median_period = listing_periods.median()
        short_periods = (listing_periods < 30).sum()
        short_periods_pct = (short_periods / len(listing_periods)) * 100

        print(f"   • Median observation period: {median_period:.0f} days (Etap 1: below a year)")
        print(f"   • Listings with <30 days: {short_periods:,} ({short_periods_pct:.1f}%) (Etap 1: significant portion)")

    # Data quality assessment
    print("\n🎯 Data Quality Assessment:")

    # Calculate improvement scores
    improvements = []
    concerns = []

    if complete_records > 11000:
        improvements.append("✓ More complete records available for analysis")

    if total_orphans < 75000:
        improvements.append("✓ Reduced orphan records")

    if 'listings_e2_df' in globals() and missing_neighbourhood_pct < 30:
        improvements.append("✓ Improved neighbourhood_cleansed completeness")

    if improvements:
        for improvement in improvements:
            print(f"   {improvement}")

    if concerns:
        for concern in concerns:
            print(f"   {concern}")

    print("\n📝 Next Steps:")
    print("   1. Proceed to etap2_02_data_validation_profiling.ipynb")
    print("   2. Detailed missing value analysis and schema verification")
    print("   3. Comprehensive comparison with Etap 1 findings")
    print("   4. Assessment of imputation strategy requirements")

    # Save summary to file
    summary_path = os.path.join(processed_data_dir_e2, 'data_loading_summary.txt')
    with open(summary_path, 'w') as f:
        f.write(f"Etap 2 Data Loading Summary - {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
        f.write("="*60 + "\n\n")
        f.write(f"Datasets loaded: {', '.join(validation_results['datasets_loaded'])}\n")
        f.write(f"Validation results saved to: {summary_path}\n")

    print(f"\n💾 Summary saved to: {summary_path}")

# Generate final summary
generate_summary_report()